In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/adultcensus/adult-census.csv
/kaggle/input/adultcensus/adult-census-numeric-test.csv


In [2]:
import pandas as pd
df = pd.read_csv("../input/adultcensus/adult-census.csv")
df.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# Separate Label and Features
target = "class"
label = df[target]
features = df.drop(columns = target)

In [4]:
num_columns = ["age", "capital-gain", "capital-loss", "hours-per-week"]
numeric_data = df[num_columns]

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(), LogisticRegression())

In [6]:
%%time
from sklearn.model_selection import cross_validate
cv_result = cross_validate(model, numeric_data, label, cv =5)
cv_result

CPU times: user 1.52 s, sys: 486 ms, total: 2.01 s
Wall time: 1.4 s


{'fit_time': array([0.14496398, 0.17291784, 0.17470503, 0.20715332, 0.26205468]),
 'score_time': array([0.05097938, 0.05117798, 0.05386066, 0.07136893, 0.09395146]),
 'test_score': array([0.79557785, 0.80049135, 0.79965192, 0.79873055, 0.80436118])}

In [7]:
scores = cv_result["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} +/- {scores.std():.3f}"
)

The mean cross-validation accuracy is: 0.800 +/- 0.003


When calling `cross-validate`(estimator, X, y, cv = 5) the following happpens:
    
* `X` and `y` are split five times with none overlapping test sets
* `estmator.fit` is called five times each time on a different training set
* a Python dictionary is returned containing a key/value containing a NumPy array 
     with 5 scores computed on the test sets 

The generalization performance of a scikit-learn model can be evaluated by:

* calling `fit` to train the model on the training set, predict on the test set to get the predictions, 
  and compute the score by passing the predictions and the true target values to some metric function
* calling `fit` to train the model on the training set and score to compute the score on the test set
* calling `cross_validate` by passing the model, the data and the target

`StandardScaler` transformer with the default parameter will:
* transform the features so that they have similar ranges 
* transforms feature values that were originally positive-only 
  into values that can be negative or positive 
* can help logistic regression converge faster (fewer iterations) 

Cross-validation allows us to:
* measure the generalization performance
* estimate the variability of the generalization score

`make_pipeline` (as well as `Pipeline`) is used to:
* combine one or more transformers and a predictor